In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import sys
import os

from skimage.morphology import binary_erosion, binary_dilation, disk
from skimage.measure import label

sys.path.append('/Users/Michael/projects/shape_sharing/src/')
from common import paths, voxel_data, images, parameters, scene, mesh

In [6]:
ms = mesh.Mesh()
ms.vertices = np.random.rand(100, 3)
ms.faces = np.random.randint(0, 100, (50, 3))
labels = np.random.randint(0, 2, (100))
print labels

In [9]:
ms.write_to_obj('./coloured_verts.obj', labels)

In [15]:
'''
make predictions for all of bigbird dataset
using my algorhtm
saves each prediction to disk
'''
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cPickle as pickle
import sys
import os
sys.path.append(os.path.expanduser("~/projects/shape_sharing/src/"))
from time import time
import copy

from common import paths
from common import parameters
from common import voxlets
from common import scene


# loading model
with open(paths.RenderedData.voxlet_model_oma_path, 'rb') as f:
    model = pickle.load(f)

# for count, tree in enumerate(model.forest.trees):
#     print count, len(tree.leaf_nodes())
#     lengths = np.array([len(leaf.exs_at_node) for leaf in tree.leaf_nodes()])
#     print np.sum(lengths<10), np.sum(np.logical_and(lengths>10, lengths<50)), np.sum(lengths>50)


test_types = ['oma']

print "Checking results folders exist, creating if not"
for test_type in test_types + ['partial_tsdf', 'visible_voxels']:
    print test_type
    folder_save_path = \
        paths.RenderedData.voxlet_prediction_path % (test_type, '_')
    folder_save_path = os.path.dirname(folder_save_path)
    print folder_save_path
    if not os.path.exists(folder_save_path):
        os.makedirs(folder_save_path)


print "MAIN LOOP"
# Note: if parallelising, should either do here (at top level) or at the
# bottom level, where the predictions are accumulated (although this might be)
# better off being GPU...)

sequence = paths.RenderedData.test_sequence()[0]

sc = scene.Scene()
sc.load_sequence(sequence, frame_nos=0, segment_with_gt=True, save_grids=False)
sc.santity_render(save_folder='/tmp/')


test_type = 'oma'

print "-> Reconstructing with oma forest"
rec = voxlets.Reconstructer(
    reconstruction_type='kmeans_on_pca', combine_type='modal_vote')
rec.set_model(model)
rec.set_scene(sc)
rec.sample_points(parameters.VoxletPrediction.number_samples)
rec.initialise_output_grid(gt_grid=sc.gt_tsdf)

blank_rec = copy.deepcopy(rec)

# Here just complete one voxlet at a time...
sampled_idxs = rec.sampled_idxs
for count, idx in enumerate(sampled_idxs[:1]):

    rec = copy.deepcopy(blank_rec)

    rec.sampled_idxs = [idx]
    prediction = rec.fill_in_output_grid_oma( render_type=[], #['matplotlib'],
        render_savepath='/tmp/renders/')
    prediction_keeping_exisiting = rec.keeping_existing

    # Rendering the prediction
    renderpath = paths.RenderedData.voxlet_prediction_img_path % \
        (test_type, sequence['name'] + '_voxlet_number_%d' % count)

In [18]:
ms1 = mesh.Mesh()
ms1.from_volume(prediction, 0)
ms1.remove_nan_vertices()

ms2 = mesh.Mesh()
ms2.from_volume(prediction_keeping_exisiting, 0)
ms2.remove_nan_vertices()

In [25]:
plt.figure(figsize=(10, 10))
plt.plot(ms2.vertices[:, 0], ms2.vertices[:, 1], 'r.')
plt.plot(ms1.vertices[:, 0], ms1.vertices[:, 1], 'b.')
#plt.axis('equal')
plt.xlim((0.4, 0.5))
plt.ylim((0.4, 0.5))